In [ ]:
! pip install datasets transformers rouge-score nltk evaluate accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 64.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 64.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.4/269.4 

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [ ]:
import transformers

print(transformers.__version__)

4.28.1


In [ ]:
model_checkpoint = "LucasThil/T5_large2_512_128"

In [ ]:
from datasets import load_dataset, load_metric

raw_datasets = load_dataset("LucasThil/miniwob_plusplus_T5_randomized_ref2")
#metric = load_metric("rouge")

raw_datasets

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/60321 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/LucasThil___parquet/LucasThil--miniwob_plusplus_T5_randomized_ref2-eda9346d2d71706b/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['history_episodes', 'instruction', 'html_snippets', 'actions', 'refs', 'keydown_texts'],
        num_rows: 60321
    })
})

In [ ]:
# select how much of the datset we want to use
coef = 0.5
raw_datasets['train'] = raw_datasets['train'].shuffle()
raw_datasets['train'] = raw_datasets["train"].select(range(int(len(raw_datasets["train"])*coef)))
raw_datasets = raw_datasets["train"].train_test_split(train_size=0.9, seed=20)
raw_datasets["validation"] = raw_datasets.pop("test")
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['history_episodes', 'instruction', 'html_snippets', 'actions', 'refs', 'keydown_texts'],
        num_rows: 27144
    })
    validation: Dataset({
        features: ['history_episodes', 'instruction', 'html_snippets', 'actions', 'refs', 'keydown_texts'],
        num_rows: 3016
    })
})

In [ ]:
# Add extra column regarding condensed target_labels
# We append together the (action, ref, text)
# Where action is either of type 'click' or 'type'
# ref is the targeted element number to perform the action on
# and type is the text content in case the action is 'type'
def condensed_targets(dataset, label):
  condensed_targets = []
  for row in dataset[label]:
    target_text = row['keydown_texts']
    if target_text is None:
      target_text = ""
    target = '{' + row['actions'] + ', ' + str(row['refs']) + ', ' + target_text + '}'
    condensed_targets.append(target)
  return condensed_targets

raw_datasets['train'] = raw_datasets['train'].add_column("targets", condensed_targets(raw_datasets, 'train'))
raw_datasets['train'] = raw_datasets['train'].remove_columns(['actions', 'refs', 'keydown_texts', 'instruction', 'html_snippets'])
raw_datasets['validation'] = raw_datasets['validation'].add_column("targets", condensed_targets(raw_datasets, 'validation'))
raw_datasets['validation'] = raw_datasets['validation'].remove_columns(['actions', 'refs', 'keydown_texts', 'instruction', 'html_snippets'])

Flattening the indices:   0%|          | 0/27144 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/3016 [00:00<?, ? examples/s]

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['history_episodes', 'targets'],
        num_rows: 27144
    })
    validation: Dataset({
        features: ['history_episodes', 'targets'],
        num_rows: 3016
    })
})

In [ ]:
print(raw_datasets['train']['history_episodes'][0])
print(raw_datasets['train']['targets'][0])

{click,11,}{click,24,}{click,17,} Select eXh, qw3H0R, vWXrcX, t4Z, gP, CQd, Y7, GUF6rCL, pB973Mb, kc and click Submit.  <body ref="14"><div id="wrap" ref="20"><div id="area" ref="1"><div id="boxes-left" ref="5"><label ref="21"><input type="checkbox" id="ch0" ref="11" value="true"></input><t ref="-254" text="exh"></t></label><label ref="4"><input type="checkbox" id="ch1" ref="24" value="true"></input><t ref="-255" text="qw3h0r"></t></label><label ref="19"><input type="checkbox" id="ch2" ref="9" value="false"></input><t ref="-256" text="kc"></t></label><label ref="7"><input type="checkbox" id="ch3" ref="10" value="true"></input><t ref="-257" text="t4z"></t></label><label ref="3"><input type="checkbox" id="ch4" ref="2" value="false"></input><t ref="-258" text="guf6rcl"></t></label><label ref="16"><input type="checkbox" id="ch5" ref="15" value="false"></input><t ref="-259" text="y7"></t></label></div><div id="boxes-right" ref="23"><label ref="6"><input type="checkbox" id="ch6" ref="18" val

In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(raw_datasets["train"])

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
tokenizer

T5TokenizerFast(name_or_path='LucasThil/T5_large2_512_128', vocab_size=32100, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra

In [ ]:
max_input_length = 512
max_target_length = 128

def preprocess_function(examples):
    inputs = [doc for doc in examples["history_episodes"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    #with tokenizer.as_target_tokenizer():
    labels = tokenizer(examples["targets"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
preprocess_function(raw_datasets['train'][:2])

{'input_ids': [[3, 2, 12570, 6, 4347, 2, 12570, 6, 1808, 6, 2, 12570, 6, 11116, 2, 12570, 6, 1714, 6, 2, 6185, 6654, 6, 3, 52, 23, 1298, 6, 549, 3698, 1298, 6, 3, 40, 357, 476, 17, 940, 6, 3, 29, 927, 7, 345, 7, 89, 6, 584, 1824, 6, 3, 89, 476, 9, 683, 566, 6, 3, 9968, 536, 5211, 11, 1214, 3325, 1538, 5, 3, 2, 6965, 6273, 17592, 20364, 3155, 2, 8481, 3, 23, 26, 17592, 210, 5846, 121, 6273, 17592, 2534, 121, 3155, 2, 8481, 3, 23, 26, 17592, 498, 121, 6273, 17592, 2294, 121, 3155, 2, 8481, 3, 23, 26, 17592, 2689, 15, 7, 18, 17068, 121, 6273, 17592, 2596, 121, 3155, 2, 40, 10333, 6273, 17592, 2368, 121, 3155, 2, 77, 2562, 686, 17592, 10031, 2689, 121, 3, 23, 26, 17592, 524, 632, 121, 6273, 17592, 2688, 121, 701, 17592, 89, 5405, 15, 121, 3155, 2, 87, 77, 2562, 3155, 2, 17, 6273, 17592, 18, 25449, 121, 1499, 17592, 208, 1824, 121, 3155, 2, 87, 17, 3155, 2, 87, 40, 10333, 3155, 2, 40, 10333, 6273, 17592, 948, 121, 3155, 2, 77, 2562, 686, 17592, 10031, 2689, 121, 3, 23, 26, 17592, 524, 536, 

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/27144 [00:00<?, ? examples/s]

Map:   0%|          | 0/3016 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

#model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

# new:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_checkpoint)

In [ ]:
import evaluate

rouge = evaluate.load("rouge")

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained('t5-large') # model_checkpoint

In [ ]:
trainer = ""
import gc
import torch
torch.cuda.empty_cache()
gc.collect()

0

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="T5_large3_512_128",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    auto_find_batch_size=True,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,
    predict_with_generate=True,
    fp16=False,
    push_to_hub=False,

    dataloader_drop_last=True,
    warmup_steps=100,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


In [ ]:
trainer.push_to_hub()

Upload file pytorch_model.bin:   0%|          | 1.00/2.75G [00:00<?, ?B/s]

Upload file runs/Apr20_16-21-37_7eef28da8b73/events.out.tfevents.1682007705.7eef28da8b73.2424.4:   0%|        …

To https://huggingface.co/LucasThil/T5_large2_512_128
   f646352..99b19ac  main -> main

   f646352..99b19ac  main -> main

To https://huggingface.co/LucasThil/T5_large2_512_128
   99b19ac..f2eca51  main -> main

   99b19ac..f2eca51  main -> main



'https://huggingface.co/LucasThil/T5_large2_512_128/commit/99b19ac4ee07228c24afb5cd7fc564d2a0f8ef71'

In [ ]:
import gc
import torch
torch.cuda.empty_cache()
gc.collect()

254

# old:

In [ ]:
batch_size = 8
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-large_rouge_random_ref_2.0",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
    optim='adamw_torch',
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
import nltk
nltk.download('punkt')
import numpy as np

# old:
def compute_metrics_old(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


ValueError: ignored

In [ ]:
trainer.push_to_hub()

Saving model checkpoint to t5-small-T5_small_test1_miniwob
Configuration saved in t5-small-T5_small_test1_miniwob/config.json
Configuration saved in t5-small-T5_small_test1_miniwob/generation_config.json
Model weights saved in t5-small-T5_small_test1_miniwob/pytorch_model.bin
tokenizer config file saved in t5-small-T5_small_test1_miniwob/tokenizer_config.json
Special tokens file saved in t5-small-T5_small_test1_miniwob/special_tokens_map.json


Upload file pytorch_model.bin:   0%|          | 32.0k/231M [00:00<?, ?B/s]

Upload file runs/Mar03_22-57-41_bbf316204f4d/events.out.tfevents.1677884320.bbf316204f4d.389.0: 100%|#########…

Upload file training_args.bin: 100%|##########| 3.56k/3.56k [00:00<?, ?B/s]

Upload file runs/Mar03_22-57-41_bbf316204f4d/1677884320.5831966/events.out.tfevents.1677884320.bbf316204f4d.38…

Upload file runs/Mar03_22-57-41_bbf316204f4d/1677884418.610464/events.out.tfevents.1677884418.bbf316204f4d.389…

remote: Scanning LFS files of refs/heads/main for validity...        
remote: LFS file scan complete.        
To https://huggingface.co/LucasThil/t5-small-T5_small_test1_miniwob
   cb44621..475fbce  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/LucasThil/t5-small-T5_small_test1_miniwob
   cb44621..475fbce  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Sequence-to-sequence Language Modeling', 'type': 'text2text-generation'}}
To https://huggingface.co/LucasThil/t5-small-T5_small_test1_miniwob
   475fbce..9a38be8  main -> main

   475fbce..9a38be8  main -> main



'https://huggingface.co/LucasThil/t5-small-T5_small_test1_miniwob/commit/475fbced79113acc22dbde1bdbca9e71e1a2fb44'

In [ ]:
import gc
import torch
torch.cuda.empty_cache()
gc.collect()

54302